In [241]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import (StandardScaler, OneHotEncoder)
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

from sklearn.metrics import (
    accuracy_score,
    classification_report,
    roc_auc_score,
    confusion_matrix
)

import plotly.express as px
import plotly.graph_objects as go

X = df.drop

In [242]:
df = pd.read_csv(
    filepath_or_buffer="heart_disease_dataset.csv"
)

In [243]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Age                      1000 non-null   int64 
 1   Gender                   1000 non-null   object
 2   Cholesterol              1000 non-null   int64 
 3   Blood Pressure           1000 non-null   int64 
 4   Heart Rate               1000 non-null   int64 
 5   Smoking                  1000 non-null   object
 6   Alcohol Intake           660 non-null    object
 7   Exercise Hours           1000 non-null   int64 
 8   Family History           1000 non-null   object
 9   Diabetes                 1000 non-null   object
 10  Obesity                  1000 non-null   object
 11  Stress Level             1000 non-null   int64 
 12  Blood Sugar              1000 non-null   int64 
 13  Exercise Induced Angina  1000 non-null   object
 14  Chest Pain Type          1000 non-null   

In [244]:
df.head()

,Age,Gender,Cholesterol,Blood Pressure,Heart Rate,Smoking,Alcohol Intake,Exercise Hours,Family History,Diabetes,Obesity,Stress Level,Blood Sugar,Exercise Induced Angina,Chest Pain Type,Heart Disease
0,75,Female,228,119,66,Current,Heavy,1,No,No,Yes,8,119,Yes,Atypical Angina,1
1,48,Male,204,165,62,Current,NaN,5,No,No,No,9,70,Yes,Typical Angina,0
2,53,Male,234,91,67,Never,Heavy,3,Yes,No,Yes,5,196,Yes,Atypical Angina,1
3,69,Female,192,90,72,Current,NaN,4,No,Yes,No,7,107,Yes,Non-anginal Pain,0
4,62,Female,172,163,93,Never,NaN,6,No,Yes,No,2,183,Yes,Asymptomatic,0


In [245]:
df["Alcohol Intake"].value_counts()

Alcohol Intake
Heavy       346
Moderate    314
Name: count, dtype: int64

In [246]:
df['Alcohol Intake'] = df['Alcohol Intake'].fillna(
    value="Minimal",
)

In [247]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Age                      1000 non-null   int64 
 1   Gender                   1000 non-null   object
 2   Cholesterol              1000 non-null   int64 
 3   Blood Pressure           1000 non-null   int64 
 4   Heart Rate               1000 non-null   int64 
 5   Smoking                  1000 non-null   object
 6   Alcohol Intake           1000 non-null   object
 7   Exercise Hours           1000 non-null   int64 
 8   Family History           1000 non-null   object
 9   Diabetes                 1000 non-null   object
 10  Obesity                  1000 non-null   object
 11  Stress Level             1000 non-null   int64 
 12  Blood Sugar              1000 non-null   int64 
 13  Exercise Induced Angina  1000 non-null   object
 14  Chest Pain Type          1000 non-null   

In [248]:
def convert_to_lowercase(df: pd.DataFrame):
    df_copy = df.copy()
    columns = df.columns

    mapper = {
        column: column.lower().replace(" ", "_") for column in columns
    }

    df_copy = df_copy.rename(mapper=mapper, axis="columns")
    return df_copy

In [249]:
df = convert_to_lowercase(df=df)

In [250]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   age                      1000 non-null   int64 
 1   gender                   1000 non-null   object
 2   cholesterol              1000 non-null   int64 
 3   blood_pressure           1000 non-null   int64 
 4   heart_rate               1000 non-null   int64 
 5   smoking                  1000 non-null   object
 6   alcohol_intake           1000 non-null   object
 7   exercise_hours           1000 non-null   int64 
 8   family_history           1000 non-null   object
 9   diabetes                 1000 non-null   object
 10  obesity                  1000 non-null   object
 11  stress_level             1000 non-null   int64 
 12  blood_sugar              1000 non-null   int64 
 13  exercise_induced_angina  1000 non-null   object
 14  chest_pain_type          1000 non-null   

In [251]:
df["smoking"].value_counts()

smoking
Never      338
Current    336
Former     326
Name: count, dtype: int64

### Analysis / Aggregation

In [252]:
df.head()

,age,gender,cholesterol,blood_pressure,heart_rate,smoking,alcohol_intake,exercise_hours,family_history,diabetes,obesity,stress_level,blood_sugar,exercise_induced_angina,chest_pain_type,heart_disease
0,75,Female,228,119,66,Current,Heavy,1,No,No,Yes,8,119,Yes,Atypical Angina,1
1,48,Male,204,165,62,Current,Minimal,5,No,No,No,9,70,Yes,Typical Angina,0
2,53,Male,234,91,67,Never,Heavy,3,Yes,No,Yes,5,196,Yes,Atypical Angina,1
3,69,Female,192,90,72,Current,Minimal,4,No,Yes,No,7,107,Yes,Non-anginal Pain,0
4,62,Female,172,163,93,Never,Minimal,6,No,Yes,No,2,183,Yes,Asymptomatic,0


In [253]:
gender_amount = (
    df
    .groupby(by="gender")
    .agg(
        total=("gender", "count"), 
        heart_diseases=("heart_disease", lambda x: (x == 1).sum()),
        no_heart_disease=("heart_disease", lambda x: (x != 1).sum()),
    )
)

gender_amount

,total,heart_diseases,no_heart_disease
gender,,,
Female,503,190,313
Male,497,202,295


In [254]:
gender_smoking = (
    df
    .groupby(by="gender")
    .agg(
        total=("gender", "count"), 
        smoker=("smoking", lambda x: (x == "Current").sum()),
        former_smoker = ("smoking", lambda x: (x == "Former").sum()),
        not_smoker=("smoking", lambda x: (x == "Never").sum()),
    )
)

gender_smoking

,total,smoker,former_smoker,not_smoker
gender,,,,
Female,503,167,159,177
Male,497,169,167,161


In [269]:
gender_alcohol_intake = (
    df
    .groupby(by="gender")
    .agg(
        total=("gender", "count"), 
        heavy=("alcohol_intake", lambda x: (x == "Heavy").sum()),
        moderate = ("alcohol_intake", lambda x: (x == "Moderate").sum()),
        minimal=("alcohol_intake", lambda x: (x == "Minimal").sum()),
    )
)

gender_alcohol_intake

,total,heavy,moderate,minimal
gender,,,,
Female,503,163,152,188
Male,497,183,162,152


### Building a Model

In [255]:
X = df.drop(labels="heart_disease", axis="columns")
y = df["heart_disease"]

In [256]:
numerical_cols = X.select_dtypes(include="int64").columns
categorical_cols = X.select_dtypes(include="object").columns

In [257]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numerical_cols),
        ("cat", OneHotEncoder(drop="first", handle_unknown="ignore"), categorical_cols)
    ]
)

In [258]:
pipeline = Pipeline(
    steps=[
        ("preprocess", preprocessor),
        ("logreg", LogisticRegression(
            max_iter=1000,
            solver="lbfgs"
        ))
    ]
)

In [259]:
X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    test_size=0.2, 
    random_state=42,
    stratify=y
)

In [260]:
pipeline.fit(X_train, y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocess', ...), ('logreg', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers con

In [261]:
feature_names = (
    pipeline.named_steps["preprocess"]
    .get_feature_names_out()
)

coeffs = pipeline.named_steps["logreg"].coef_[0]

coef_df = (
    pd.DataFrame(
        {
            "feature": feature_names,
            "coefficient": coeffs
        }
    ).sort_values(by="coefficient", ascending=False)
)

In [262]:
coef_df

,feature,coefficient
0,num__age,3.001530
1,num__cholesterol,2.012145
13,cat__diabetes_Yes,0.387047
7,cat__gender_Male,0.334633
12,cat__family_history_Yes,0.309881
15,cat__exercise_induced_angina_Yes,0.169339
18,cat__chest_pain_type_Typical Angina,0.168716
16,cat__chest_pain_type_Atypical Angina,0.153992
14,cat__obesity_Yes,0.076563
3,num__heart_rate,0.030456


In [263]:
top10 = (
    coef_df
    .assign(abs_coef=lambda d: d["coefficient"].abs())
    .sort_values("abs_coef", ascending=False)
    .head(10)
    .sort_values("coefficient")
)

fig = px.bar(
    top10,
    x="feature",
    y="coefficient",
    text=top10["coefficient"].round(3),
    title="Top 10 Logistic Regression Coefficients (Heart Disease)",
    template="plotly_dark"
)

fig.update_traces(
    textposition="outside"
)

fig.update_layout(
    xaxis_title="Feature",
    yaxis_title="Coefficient (Log Odds)",
    xaxis_tickangle=45,
    yaxis=dict(zeroline=True, zerolinewidth=2),
    height=600,
    margin=dict(l=60, r=40, t=80, b=120)
)

fig.show()

### Model Evaluation

In [264]:
y_pred = pipeline.predict(X_test)
y_proba = pipeline.predict_proba(X_test)[:, 1]

In [265]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_proba))
print(classification_report(y_test, y_pred))

Accuracy: 0.86
ROC AUC: 0.9511349306431274
              precision    recall  f1-score   support

           0       0.87      0.91      0.89       122
           1       0.85      0.78      0.81        78

    accuracy                           0.86       200
   macro avg       0.86      0.85      0.85       200
weighted avg       0.86      0.86      0.86       200



In [266]:
cm = confusion_matrix(y_test, y_pred)

correct_mask = np.eye(cm.shape[0], dtype=bool)

color_matrix = correct_mask.astype(int)

fig = go.Figure(
    data=go.Heatmap(
        z=color_matrix,
        x=["Predicted No Disease", "Predicted Disease"],
        y=["No Disease", "Disease"],
        text=cm,
        texttemplate="%{text}",
        textfont={"size": 18},
        colorscale=[
            [0, "#c0392b"],  # wrong → red
            [1, "#27ae60"]   # right → green
        ],
        showscale=False
    )
)

fig.update_layout(
    title="Confusion Matrix – Correct vs Wrong Predictions",
    template="plotly_dark",
    xaxis_title="Prediction",
    yaxis_title="Actual",
    height=500
)

fig.show()